In [1]:
import pandas as pd
import geopandas as gpd

# Importing data and setting green threshold

In [2]:
df_raw = gpd.read_file("../raw_data/project_data.shp")

In [3]:
# We decide to call a block green when more than 15% of its area is covered
# by green roofs

green_thresshold = 15
df_raw['green_roof'] = (df_raw['gruen20_p']>green_thresshold).astype(int)

In [4]:
# We see that our data is unbalanced and therefore we need to be carefull
# with the scoring metric we use.
df_raw['green_roof'].value_counts()/len(df_raw)

green_roof
0    0.981725
1    0.018275
Name: count, dtype: float64

In [9]:
df_raw.columns

Index(['typ__gr', 'typklar__g', 'gruen20_m2', 'gint20_m2', 'gex20_m2',
       'gruen20_p', 'gint20_p', 'gex20_p', 'area_geb', 'anzahl_gru',
       'anzahl_geb', 'gruen16_m2', 'gruen16_p', 'diff_20_16', 'schl5', 'bez',
       'bezirk', 'woz', 'woz_name', 'ststrnr', 'ststrname', 'typ__fln', 'nutz',
       'nutzung', 'flalle', 'ueberw_dek', 'freistehen', 'doppelhaus',
       'gereihtes', 'anderertyp', 'x_bis_1900', 'x1901_1910', 'x1911_1920',
       'x1921_1930', 'x1931_1940', 'x1941_1950', 'x1951_1960', 'x1961_1970',
       'x1971_1980', 'x1981_1990', 'x1991_2000', 'x2001_2010', 'x2011_2015',
       'ew2015', 'pl_id', 'air_pollut', 'thermal_st', 'status_num',
       'status_val', 'dyn_val', 'index__ren', 'bez_rent_a', 'rent',
       'unemp_bene', 'social_hou', 'city_owned', 'rent_durat', 'aparts_sol',
       'plz', 'subsidized', 'geometry', 'green_roof'],
      dtype='object')

# Cleaning

In [5]:
def preprocess_dataframe(df):
    """
    In this function we drop columns, rename columns from german to english,
    convert column types to the correct ones, drop NaNs for some columns and convert 
    percentages to decimals.
    """
    # Drop columns to only keep the ones we want to use
    df = df_raw.drop(columns=['schl5','typ__fln','nutz','nutzung','ststrname','ststrnr','pl_id',
                      'index__ren','gruen20_m2','gruen20_p','typ__gr','bez','woz',
                      'status_num','bez_rent_a','plz','x1901_1910', 'x1911_1920',
                      'x1921_1930','x1931_1940','x1941_1950', 'x1951_1960', 'x1961_1970',
                      'x1971_1980','x1981_1990','x1991_2000', 'x2001_2010', 'x2011_2015',
                      'x_bis_1900','gint20_m2','gex20_m2','gint20_p','gex20_p','freistehen',
                      'doppelhaus','gereihtes','anderertyp','gruen16_m2','gruen16_p','diff_20_16',
                      'area_geb','anzahl_gru','anzahl_geb','flalle','ueberw_dek'])
    
    # Rename all the columns to their new value
    df.rename(columns = {'typklar__g':'usetype_block','bezirk':'district', 'woz_name':'built_type',
                         'ew2015':'residents', 'air_pollut':'air_pollution','aparts_sol':'aparts_sold',
                         'thermal_st':'thermal_stress','status_val':'social_status','dyn_val':'social_dyn',
                         'unemp_bene':'unemp_benef','city_owned':'hous_assoc', 'rent_durat':'rent_duration'}
                      , inplace = True)
    
    # Convert some column dtypes to their correct Dtype -> (From object to float32)
    df[['rent','unemp_benef','social_hou',
        'hous_assoc','rent_duration','aparts_sold']] = df[['rent','unemp_benef','social_hou','hous_assoc',
                                               'rent_duration','aparts_sold']].astype('float32')
    
    # Convert float64 columns into float32
    float64_cols = df.select_dtypes(include=['float64']).columns
    df[float64_cols] = df[float64_cols].astype('float32')
    
    # Convert int64 columns into bool. For 'green_roof' and 'subsidized' features
    int64_cols = df.select_dtypes(include=['int64']).columns
    df[int64_cols] = df[int64_cols].astype('bool')
    
    # Delete rows with NaN in 'built_type' column
    df = df[df['built_type'].isna()==False]
    
    # Delete rows where the rent is == 0. They give no useful information
    df = df[df['rent']!=0]
    
    # Get percentages from the columns
    df[['unemp_benef','social_hou','hous_assoc','rent_duration']] = df[['unemp_benef','social_hou','hous_assoc','rent_duration']] /100 
    
    return df

In [6]:
df = preprocess_dataframe(df_raw)

# Exporting

In [8]:
df.to_file("../raw_data/project_data_clean.shp")

/var/folders/ly/4bxnth7x7_b57zj_8nf4ygvr0000gn/T/ipykernel_21576/3752511159.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  df.to_file("../raw_data/project_data_clean.shp")
